In [1]:
!pip install pybullet imageio-ffmpeg

import os
import time
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import cv2
import imageio_ffmpeg
from base64 import b64encode
from IPython.display import HTML
import numpy as np
import imageio_ffmpeg
import pybullet as p
import math


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 6.3 MB/s eta 0:00:00


In [2]:
# Configura la simulazione e la fotocamera
p.connect(p.DIRECT)  # Usa la modalità senza GUI per evitare un impatto sulle prestazioni
p.resetSimulation() # Lui va chiamato la mattina appena apri il computer
p.setGravity(0, 0, -10)
p.setTimeStep(0.001)  # Imposta il passo temporale per la simulazione

# Aggiungi il percorso dei dati PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [3]:
!git clone https://github.com/Gaianeve/Real_Mega_Fufi.git
%cd Real_Mega_Fufi/Robot/PIPPA

Cloning into 'Real_Mega_Fufi'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 778 (delta 193), reused 25 (delta 25), pack-reused 557 (from 1)
Receiving objects: 100% (778/778), 7.76 MiB | 10.96 MiB/s, done.
Resolving deltas: 100% (506/506), done.
/content/Real_Mega_Fufi/Robot/PIPPA


In [4]:
useFixedBase = False
flags = p.URDF_INITIALIZE_SAT_FEATURES

plane_pos = [0,0,-0.1]
plane = p.loadURDF("plane.urdf", plane_pos, flags = flags, useFixedBase=useFixedBase)

# position of the base of the model
PIP_position = [0,0,1.043]
orientation  = [np.pi,0,0]
PIP_orientation = p.getQuaternionFromEuler(orientation)
PIPPA_id = p.loadURDF("PIPPA.urdf",basePosition = PIP_position,baseOrientation = PIP_orientation,\
                      useFixedBase=useFixedBase)


In [5]:
link_name_to_index = {p.getBodyInfo(PIPPA_id)[0].decode('UTF-8'):-1,}

for id in range(p.getNumJoints(PIPPA_id)):
  name = p.getJointInfo(PIPPA_id, id)[12].decode('UTF-8')
  link_name_to_index[name] = id

print(link_name_to_index)

{'Top_core_Link': -1, 'Top_Link_1': 0, 'Cover_Link_1': 1, 'Gamba_Link_1': 2, 'Elastic_Joint_Link_1': 3, 'Colonna_Link_1': 4, 'Bottom_Link_1': 5, 'Top_Link_2': 6, 'Cover_Link_2': 7, 'Gamba_Link_2': 8, 'Elastic_Joint_link_2': 9, 'Colonna_Link_2': 10, 'Bottom_Link_2': 11, 'Top_Link_3': 12, 'Cover_Link_3': 13, 'Gamba_Link_3': 14, 'Elastic_Joint_link_3': 15, 'Colonna_Link_3': 16, 'Bottom_Link_3': 17}


In [8]:
# AL 6082 (Alluminio)
lateralFriction_Al = 0.3
restitution_Al = 0.2
rollingFriction_Al = 0.001
spinningFriction_Al = 0.01
linearDamping_Al = 0.04
angularDamping_Al = 0.1
contactStiffness_Al = 3e6
contactDamping_Al = 800

# AISI 304 (Acciaio Inox)
lateralFriction_AISI = 0.4
restitution_AISI = 0.1
rollingFriction_AISI = 0.001
spinningFriction_AISI= 0.05
linearDamping_AISI = 0.03
angularDamping_AISI = 0.1
contactStiffness_AISI = 8e6
contactDamping_AISI = 1500

# Acciaio Maraging (Maragià per gli amici)
lateralFriction_Mar = 0.3
restitution_Mar = 0.2
rollingFriction_Mar = 0.001
spinningFriction_Mar = 0.02
linearDamping_Mar = 0.04
angularDamping_Mar = 0.15
contactStiffness_Mar = 3e6
contactDamping_Mar = 800

link_indices_Al = [link_name_to_index['Bottom_Link_1'],link_name_to_index['Bottom_Link_2'],\
                   link_name_to_index['Bottom_Link_3'], link_name_to_index['Gamba_Link_1'],\
                   link_name_to_index['Gamba_Link_2'], link_name_to_index['Gamba_Link_3'],\
                   link_name_to_index['Top_core_Link'], link_name_to_index['Cover_Link_1'], \
                   link_name_to_index['Cover_Link_2'], link_name_to_index['Cover_Link_3']]

link_indices_AISI = [link_name_to_index['Top_Link_2'], link_name_to_index['Top_Link_3'],\
                     link_name_to_index['Top_Link_1'], link_name_to_index['Colonna_Link_1'],\
                     link_name_to_index['Colonna_Link_2'],link_name_to_index['Colonna_Link_3']]

link_indices_Mar = [link_name_to_index['Elastic_Joint_Link_1'], link_name_to_index['Elastic_Joint_link_2'],\
                    link_name_to_index['Elastic_Joint_link_3']]

# Al 6082
for link in link_indices_Al:
  p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_Al,\
                   restitution=restitution_Al, \
                   spinningFriction= spinningFriction_Al, rollingFriction= rollingFriction_Al,\
                   linearDamping=linearDamping_Al, angularDamping=angularDamping_Al,\
                   contactStiffness = contactStiffness_Al,contactDamping = contactDamping_Al)
# AISI 304
for link in link_indices_AISI:
  p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_AISI,\
                   restitution=restitution_AISI, \
                   spinningFriction= spinningFriction_AISI, rollingFriction= rollingFriction_AISI,\
                   linearDamping=linearDamping_AISI, angularDamping=angularDamping_AISI,\
                   contactStiffness = contactStiffness_AISI, contactDamping = contactDamping_AISI)
# Maragià
for link in link_indices_AISI:
  p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_Mar,\
                   restitution=restitution_Mar, \
                   spinningFriction= spinningFriction_Mar, rollingFriction= rollingFriction_Mar,\
                   linearDamping=linearDamping_Mar, angularDamping=angularDamping_Mar,\
                   contactStiffness = contactStiffness_Mar, contactDamping = contactDamping_Mar)

In [13]:

# Parametri della fotocamera
cam_target_pos = [0, 0, 0]
cam_distance = 2.0
cam_yaw, cam_pitch, cam_roll = 50, -30.0, 0
cam_width, cam_height = 480, 360
cam_up_axis_idx = 2
cam_fov = 60
cam_near_plane, cam_far_plane = 0.01, 100

# Imposta la registrazione del video
vid = imageio_ffmpeg.write_frames('simulation_output.mp4', (cam_width, cam_height), fps=30)
vid.send(None)  # Inizializza con un fotogramma vuoto

# Parametri per la simulazione
simulation_time = 180
sampling_frequency = 1 / 100
num_points = int(simulation_time * sampling_frequency)

# Liste per registrare le posizioni
leg_positions_x_1, leg_positions_x_2, leg_positions_x_3 = [], [], []
leg_positions_y_1, leg_positions_y_2, leg_positions_y_3 = [], [], []
leg_positions_z_1, leg_positions_z_2, leg_positions_z_3 = [], [], []
# Forza esterna da applicare
force_module = np.array([50, 0, 0])
force_position = np.array([0, 0, 0.99])  # Posizione dove applicare la forza

In [ ]:

# Ciclo principale della simulazione
for step in range(1000):
    p.stepSimulation()  # Avanza di un passo nella simulazione

    if step == 50:
      # Applicare la forza esterna al primo passo della simulazione
      p.applyExternalForce(
          objectUniqueId=PIPPA_id,
          linkIndex=link_name_to_index['Bottom_Link_1'],
          forceObj=force_module,
          posObj=force_position,
          flags=p.WORLD_FRAME
      )

      p.applyExternalForce(
          objectUniqueId=PIPPA_id,
          linkIndex=link_name_to_index['Bottom_Link_2'],
          forceObj=force_module,
          posObj=force_position,
          flags=p.WORLD_FRAME
      )

      p.applyExternalForce(
          objectUniqueId=PIPPA_id,
          linkIndex=link_name_to_index['Bottom_Link_3'],
          forceObj=force_module,
          posObj=force_position,
          flags=p.WORLD_FRAME
      )

    # Registrazione della posizione della gamba
    leg_position_1 = p.getLinkState(PIPPA_id, link_name_to_index['Gamba_Link_1'])[0]
    leg_position_2 = p.getLinkState(PIPPA_id, link_name_to_index['Gamba_Link_2'])[0]
    leg_position_3 = p.getLinkState(PIPPA_id, link_name_to_index['Gamba_Link_3'])[0]

    #append x
    leg_positions_x_1.append(leg_position_1[0])
    leg_positions_x_2.append(leg_position_2[0])
    leg_positions_x_3.append(leg_position_3[0])
    # append y
    leg_positions_y_1.append(leg_position_1[1])
    leg_positions_y_2.append(leg_position_2[1])
    leg_positions_y_3.append(leg_position_3[1])
    #append z
    leg_positions_z_1.append(leg_position_1[2])
    leg_positions_z_2.append(leg_position_2[2])
    leg_positions_z_3.append(leg_position_3[2])


    # Calcola la vista e la proiezione della fotocamera
    cam_view_matrix = p.computeViewMatrixFromYawPitchRoll(
        cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx
    )
    cam_projection_matrix = p.computeProjectionMatrixFOV(
        cam_fov, cam_width / cam_height, cam_near_plane, cam_far_plane
    )

    # Cattura l'immagine
    image = p.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]

    # Scrivi il fotogramma nel video
    vid.send(np.ascontiguousarray(image))

# Rilascia le risorse video
vid.close()

# Disconnessione dalla simulazione
p.disconnect()

In [ ]:
# Ora visualizziamo il video registrato nel notebook
mp4 = open('simulation_output.mp4', 'rb').read()  # Leggi il file video
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()  # Convertilo in Base64

# Visualizza il video nel notebook Jupyter
HTML('' % data_url)


In [ ]:
#plotting positions
fig, axs = plt.subplots(3, 1, figsize=(8, 12))  # 3 righe, 1 colonna

# x coordinate
axs[0].plot(leg_positions_x_1, label='leg1_x')
axs[0].plot(leg_positions_x_2, label='leg2_x')
axs[0].plot(leg_positions_x_3, label='leg3_x')
axs[0].legend()
axs[0].grid()
axs[0].minorticks_on()
#axs[0].set_xlim(0,4000)
axs[0].set_title('X Coordinate')

# y coordinate
axs[1].plot(leg_positions_y_1, label='leg1_y')
axs[1].plot(leg_positions_y_2, label='leg2_y')
axs[1].plot(leg_positions_y_3, label='leg3_y')
axs[1].legend()
axs[1].grid()
axs[1].minorticks_on()
#axs[1].set_xlim(0,4000)
axs[1].set_title('Y Coordinate')

# z coordinate
axs[2].plot(leg_positions_z_1, label='leg1_z')
axs[2].plot(leg_positions_z_2, label='leg2_z')
axs[2].plot(leg_positions_z_3, label='leg3_z')
axs[2].legend()
axs[2].grid()
axs[2].minorticks_on()
#axs[2].set_xlim(0,4000)
axs[2].set_title('Z Coordinate')

plt.tight_layout()  # Ottimizza la disposizione delle sottotrame
plt.show()


In [ ]:
#motion xy for 3 leg
plt.figure('xy motion')
plt.plot(leg_positions_x_1, leg_positions_y_1, label = 'leg1')
plt.plot(leg_positions_x_2, leg_positions_y_2, label = 'leg2')
plt.plot(leg_positions_x_3, leg_positions_y_3, label = 'leg3')
plt.legend()
plt.grid()
plt.minorticks_on()

plt.show()